In [1]:
from pyspark.sql import SparkSession 
spark=SparkSession.builder.master("spark://localhost:7077").getOrCreate()

In [9]:
df=spark.read.parquet("hdfs://172.17.0.2:9000/nifi/*.parquet")

In [15]:
##Ejercicio 3.1 Mostrar los resultados siguientes 
##VendorId, Tpep_pickup_datetime, Total_amount donde el total_amount sea menor a 10 dolares
df.createOrReplaceTempView("yellow_tripdata")
spark.sql("Select VendorID, cast(Tpep_pickup_datetime as date), Total_amount FROM yellow_tripdata where total_amount<10").show(10) 


+--------+--------------------+------------+
|VendorID|Tpep_pickup_datetime|Total_amount|
+--------+--------------------+------------+
|       1|          2020-12-31|         4.3|
|       2|          2020-12-31|         8.3|
|       2|          2020-12-31|        9.96|
|       2|          2020-12-31|         9.3|
|       2|          2020-12-31|         5.8|
|       1|          2020-12-31|         0.0|
|       1|          2020-12-31|         9.3|
|       2|          2020-12-31|         9.8|
|       2|          2020-12-31|         8.8|
|       2|          2020-12-31|        9.96|
+--------+--------------------+------------+
only showing top 10 rows



In [20]:
##Mostrar los 10 días que más se recaudo dinero (Tpep_pickup_datetime, total_amount) 
spark.sql("Select cast(Tpep_pickup_datetime as date) as Fecha, SUM(total_amount) as total_amount FROM yellow_tripdata \
group by Fecha order by total_amount desc LIMIT 10").show()

[Stage 18:=============================>                            (1 + 1) / 2]

+----------+-----------------+
|     Fecha|     total_amount|
+----------+-----------------+
|2021-01-28|961322.5600002451|
|2021-01-22|942205.9300002148|
|2021-01-29|937373.5100002222|
|2021-01-21|932444.4500002082|
|2021-01-15|931628.1900002063|
|2021-01-14|926664.0400001821|
|2021-01-27|  895259.87000017|
|2021-01-19|890581.4500001629|
|2021-01-07|887670.1600001527|
|2021-01-08| 878002.730000146|
+----------+-----------------+



In [21]:
##Mostrar los 10 viajes que menos dinero recaudo en viajes mayores a 10 millas (Trip_distance, total_amount) 

spark.sql("Select trip_distance, total_amount FROM yellow_tripdata where trip_distance>10 order by total_amount asc LIMIT 10").show()

[Stage 21:=============================>                            (1 + 1) / 2]

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|        12.68|      -252.3|
|        34.35|     -176.42|
|        14.75|      -152.8|
|        33.96|     -127.92|
|         29.1|      -119.3|
|        26.94|      -111.3|
|        20.08|      -107.8|
|        19.55|      -102.8|
|        19.16|      -90.55|
|        25.83|      -88.54|
+-------------+------------+



In [23]:
##Mostrar los viajes de más de dos pasajeros que hayan pagado con tarjeta de crédito (mostrar solo las columnas trip_distance, tpep_pickup_datetime) 
spark.sql("Select trip_distance, cast(tpep_pickup_datetime as date) FROM yellow_tripdata where passenger_count>2 and payment_type=1").show()

+-------------+--------------------+
|trip_distance|tpep_pickup_datetime|
+-------------+--------------------+
|         6.11|          2020-12-31|
|          1.7|          2020-12-31|
|         3.15|          2020-12-31|
|        10.74|          2020-12-31|
|         2.01|          2020-12-31|
|         2.85|          2020-12-31|
|         1.68|          2020-12-31|
|         0.77|          2020-12-31|
|          0.4|          2020-12-31|
|        16.54|          2020-12-31|
|          2.3|          2020-12-31|
|          2.5|          2020-12-31|
|         1.29|          2020-12-31|
|          1.3|          2020-12-31|
|          0.8|          2020-12-31|
|         1.49|          2020-12-31|
|         1.62|          2020-12-31|
|         5.04|          2020-12-31|
|          2.5|          2020-12-31|
|         3.13|          2020-12-31|
+-------------+--------------------+
only showing top 20 rows



In [28]:
##Mostrar los 7 viajes con mayor propina en distancia mayores a 10 millas (mostrar campos tpep_pickup_datetime, trip_distance, passenger_count, tip_amount)
spark.sql("Select cast(tpep_pickup_datetime as date), trip_distance, passenger_count, tip_amount FROM yellow_tripdata where trip_distance>10 \
order by tip_amount desc LIMIT 7").show()

+--------------------+-------------+---------------+----------+
|tpep_pickup_datetime|trip_distance|passenger_count|tip_amount|
+--------------------+-------------+---------------+----------+
|          2021-01-20|        427.7|            1.0|   1140.44|
|          2021-01-03|        267.7|            1.0|     369.4|
|          2021-01-12|        326.1|            0.0|    192.61|
|          2021-01-19|        260.5|            1.0|    149.03|
|          2021-01-31|         11.1|            0.0|     100.0|
|          2021-01-01|        14.86|            2.0|      99.0|
|          2021-01-18|         13.0|            0.0|      90.0|
+--------------------+-------------+---------------+----------+



In [30]:
##Mostrar para cada uno de los valores de RateCodeID, el monto total y el monto promedio. Excluir los viajes en donde RateCodeID es Group Rate

spark.sql("Select RateCodeID, SUM(total_amount) as total_amount, AVG(total_amount) as avg_amount FROM yellow_tripdata group by RateCodeID").show()

+----------+--------------------+------------------+
|RateCodeID|        total_amount|        avg_amount|
+----------+--------------------+------------------+
|       1.0|1.9496468430212937E7|15.606626116946773|
|       4.0|   90039.93000000082| 74.90842762063296|
|      null|  3051485.7299998133| 31.02616855783119|
|       3.0|   67363.26000000043| 78.69539719626219|
|       2.0|   973635.4700000732| 65.52937609369182|
|      99.0|  1748.0699999999997| 48.55749999999999|
|       6.0|               80.11|             8.011|
|       5.0|  255075.08999999086|48.939963545662096|
+----------+--------------------+------------------+

